In [ ]:
!pip install folium

CARTO BIO CO2

In [1]:
import folium
import pandas as pd

In [32]:
# import des données / création du df
df_rebours = pd.read_excel('./data/bio_co2.xlsx', sheet_name='Rebours_')
df_injections = pd.read_excel('./data/bio_co2.xlsx', sheet_name='Injections_')

df_injections.dropna().head(3)

,Annee mise en service,Nom du site,Type de site,Commune,Lat,Lng,Code EPCI,EPCI,Departement,Region,...,Production_CO2_ktparan,Gestionnaire de registre,N° de PITD/PITP,Code Departement,Code Region,Augmentation prévue ?,﻿ID unique projet,Date de fermeture du site,Site ouvert ?,Code Commune Courant
602,2015.0,SARL SIOULE - BIOGAZ,Agricole autonome,Saint-Pourçain-sur-Sioule,46.304509,3.305378,200071389.0,CC Saint-Pourçain Sioule Limagne,Allier,Auvergne-Rhône-Alpes,...,1.2,GRTgaz,GD0455,3.0,84.0,Augmentation supplémentaire prévue,31.0,2021-07-13,False,3254.0


In [24]:
# Adjuster la position le départ et le niveau de zoom initial
# définition de la position -> Juste donner les lattitude et longitude dans un tuple(Nord, Est) -> attention si vos coordonnées sont 74°W il faut les mettre en -74°E
start_position = (48.866667 , 2.33333) #coordonnées de Paris

# Création d'une variable dans laquelle on crée une map de base avec un style graphique au choix tiles = Stamen (Terrain, toner, Watercolor) ou tiles = CartoDB(positron, dark_matter )
base_map = folium.Map(tiles ='CartoDB dark_matter', location = start_position, zoom_start= 6, )


# ajout d'un calque de contour/frontière
#layer1 = folium.GeoJson()

# ajout d'un marker
folium.Marker(location = start_position,
              tooltip='',
              icon = folium.Icon(icon ='road', prefix='fa', color='red'), # aller sur glyphicons getbootstraps par exemple pour récupérer les noms
              popup= folium.Popup('<p>toutes les infos</p>', max_width=500) 
              )#.add_to(base_map)

# logo GRT ou Designlab
from folium.plugins import FloatImage
#url = 'https://upload.wikimedia.org/wikipedia/fr/thumb/5/57/LogoGRTgaz2023.svg/1200px-LogoGRTgaz2023.svg.png'
logo = folium.CustomIcon(icon_image='./ressources/logo_designlab_proto.png', icon_size=(100, 100))
FloatImage('./ressources/logo_designlab_proto.png', bottom=2, left=1, height = '28px', width = '60px').add_to(base_map)



# ajout d'un gestionnaire de calque
# definition du style de bordure
borderStyleReg={
    "color":"grey",
    "weight" : 2,
    "fill" : False 
}

borderStyleDep={
    "color":"blue",
    "weight" : 2,
    "fill" : False 
}

# chargement des frontière des régions à partir d'un geojson
folium.GeoJson(data = './data/regions.geojson',
               name="Régions",
               style_function =lambda x: borderStyleReg,
               )#.add_to(base_map)


# chargement des frontière des départements à partir d'un geojson
folium.GeoJson(data = './data/departements.geojson',
               name="Départements",
               style_function =lambda x: borderStyleDep,
               )#.add_to(base_map)

# CALQUE REBOURS           
reboursLayer = folium.FeatureGroup('Rebours').add_to(base_map)

for i in range(len(df_rebours)):
    latVal = df_rebours.iloc[i]['Lat']
    lngVal = df_rebours.iloc[i]['Lng']
    typologieVal = df_rebours.iloc[i]['Typologie']
    debitVal = df_rebours.iloc[i]['DébitCO2_ktparan']
    colorVal = "blue" if typologieVal == 'Rebours' else "red"
    
    cRad = debitVal*100
    folium.Circle(
        location=[latVal, lngVal],
        radius= cRad,
        color = colorVal,
        weight = 2,
        fill = True
    ).add_to(reboursLayer)


# CALQUE INJECTIONS           
injectionsLayer = folium.FeatureGroup('Injections').add_to(base_map)

for i in range(len(df_injections)):
    latVal = df_injections.iloc[i]['Lat']
    lngVal = df_injections.iloc[i]['Lng']
    typologieVal = df_injections.iloc[i]['Type de site']
    debitVal = df_injections.iloc[i]['Production_CO2_ktparan']
    colorVal = "red"
    
    cRad = debitVal*100
    folium.Circle(
        location=[latVal, lngVal],
        radius= cRad,
        color = colorVal,
        weight = 2,
        fill = True
    ).add_to(injectionsLayer)



folium.LayerControl().add_to(base_map)

# affiche dans jupiter
base_map

ValueError: Location values cannot contain NaNs.

In [60]:
# sauver une map là où est l'executable
base_map.save('basemap.html')

# on peut vérifier en regardant avec la commande
!ls

# on peut l'ouvrir (va l'ouvrir dans le navigateur)
!open basemap.html

Test_folium.ipynb basemap.html      data


In [ ]:
# Création d'une variable dans laquelle on crée une map de base avec un style graphique au choix tiles = Stamen (Terrain, toner, Watercolor) ou tiles = CartoDB(positron, dark_matter )
base_map = folium.Map(tiles ='CartoDB dark_matter')

# base map est un folium map object pour vérifier on check avec type, qui contient la map entière
type(base_map)
base_map